# Simulated Data: Extremely Noisy 4-D STEM Strain Mapping Using Cycle Consistent Spatial Transforming Autoencoders 


## Package Installation

Make sure to install auto4dstem from source:

```bash
pip install auto4dstem
```

## Package Loading

In [1]:
%load_ext autoreload
%autoreload 2
import os
import torch
import pickle
import warnings
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
from cmcrameri import cm
from auto4dstem.nn.Train_Function import Train
from auto4dstem.masks.masks import Mask
from auto4dstem.viz.viz import set_format_Auto4D, visualize_simulate_result, visual_performance_plot,normalized_strain_matrices
from m3util.util.IO import download_files_from_txt

## Device Setting

In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="6"

if torch.cuda.is_available():
    device = torch.device('cuda')
    print("Using GPU")
else:
    device = torch.device('cpu')
    print("Using CPU")

Using GPU


## Visualization Format 

Set the format for standard plotting throughout the notebook

In [3]:
set_format = set_format_Auto4D()
pylab.rcParams.update(set_format)

In [4]:
# warnings.filterwarnings('ignore') 

## Download Data and Pre-Trained Models

In [5]:
folder_name = './data'
file_download =  "/home/jca92/Auto4DSTEM/content/Simulated_4dstem/Extremely_Noisy_4DSTEM_Strain_Mapping_Using_CC_ST_AE_Simulated/simulated_label_weights_affine_para"

In [17]:
# check where are you from the root path
download_files_from_txt(file_download,folder_name)

  0%|          | 0/72 [00:00<?, ?it/s]

  1%|▏         | 1/72 [00:25<29:53, 25.27s/it]

Downloaded: 40percent_noisy_simulated_4dstem_pretrained_weights.pkl


  3%|▎         | 2/72 [00:30<15:47, 13.54s/it]

Downloaded: 45percent_noisy_simulated_4dstem_pretrained_weights.pkl


...

Downloaded: simulated_4dstem_grain_index.npy


 99%|█████████▊| 71/72 [04:31<00:00,  1.15it/s]

Downloaded: MAE_list_simulated_4dstem.pickle


100%|██████████| 72/72 [12:34<00:00, 10.48s/it] 

Downloaded: polycrystal_output4D.mat


## Mask Construction



### Build Mask

We build a ring mask with inner radius 50 and outer radius 85, to only calculate the loss in the region of interest where there are diffraction spots. This avoids the central region where there is no diffraction spot - and limited information regarding the strain. 

In [6]:
# set mask class
set_mask = Mask()

# generate mask 
mask_tensor, mask_list = set_mask.mask_ring(radius_1=50, radius_2 = 85)

## Benchmark Simulated 4DSTEM Dataset

### Set BKG level, Load Data and Pre-Trained Weights

In [7]:
bkg_intensity = 0
bkg_str = format(int(bkg_intensity*100),'02d')
data_path = os.path.abspath(f'{folder_name}/polycrystal_output4D.mat')
rotation_path = f'{folder_name}/{bkg_str}Percent_pretrained_rotation.npy'

## Initialize the training class from noise-free simulated dataset

In [ ]:
tc = Train(data_path,
                device = device, 
                transpose = (1,0,3,2),
                background_weight=bkg_intensity,
                learned_rotation=rotation_path,
                adjust_learned_rotation = 0,
                num_base=1,
                up_size=800,
                scale_limit=0.05,
                shear_limit=0.1,
                rotation_limit=0.1,
                trans_limit = 0.15,
                adj_mask_para=0,
                fixed_mask = mask_list,
                check_mask = None,
                interpolate = True,
                revise_affine = False,
                folder_path = folder_name
                )

In [ ]:
tc.crop_one_image(clim=[0,4e-5])

**Figure 1: One random  diffration image from 4D-STEM**

In [ ]:
tc.visual_noise(noise_level = [0,0.25,0.6],file_name='simulated',add_label = True,save_format = 'png')

**Figure 2: Diffract image with various Poisson Ditributed Noise intensity. a) Raw data. b) Applied 10% noise intensity. c) 25% noise intensity. d) 45% noise intensity. e) 60% Noise Intensity.**

## Load pretrained weight to neural network

In [14]:
weight_path = f"{folder_name}/{bkg_str}percent_noisy_simulated_4dstem_pretrained_weights.pkl"
tc.load_pretrained_weight(weight_path)

## Generate and save results

In [ ]:
# # set batch size and load dataset into DataLoader
tc.batch_size = 256
tc.predict(train_process='2',
           save_strain = True,
           save_rotation = True,
           file_name=bkg_intensity)

## Initialize visual result class 

### Set hyper parameters of the class

In [16]:
bkg_intensity = 0
bkg_str = format(int(bkg_intensity*100),'02d')
rotation_path = f'{folder_name}/{bkg_str}Per_2_train_process_rotation.npy'
strain_path = f'{folder_name}/{bkg_str}Per_2_train_process_scale_shear.npy'
file_py4DSTEM = f'{folder_name}/analysis_bg{bkg_str}per_1e5counts__strain.h5'
rotation_clean = np.load(rotation_path)
scale_shear_clean = np.load(strain_path)

### Visualize results in notebook

In [17]:
img_show = True
if img_show:
    %matplotlib inline
else:
    plt.ioff()

### Create class and visualize rotation result

In [ ]:
visual = visualize_simulate_result(rotation =  rotation_clean,
                                scale_shear = scale_shear_clean,
                                file_py4DSTEM = file_py4DSTEM,
                                label_rotation_path = folder_name +'/Label_rotation.npy',
                                label_xx_path = folder_name +'/Label_strain_xx.npy',
                                label_yy_path = folder_name +'/Label_strain_yy.npy',
                                label_xy_path = folder_name +'/Label_shear_xy.npy',
                                cmap_rotation = cm.vikO,
                                cmap_strain= cm.vik,
                                cmap_mae=cm.vik,
                                noise_intensity = 0,
                                angle_shift = 25,
                                strain_diff_range = [-0.03,0.03],
                                strain_rotation_range = [-40,30],
                                mae_diff_range = [-0.006,0.006],
                                mae_rotation_range = [-0.5,0.5],
                                label_style= 'b'
                               )

### Visualize label map

In [ ]:
visual.visual_label_map(cmap_strain = cm.vik,
                        cmap_rotation = cm.vikO,
                        label_style='b')

### Visualize strain map result

In [ ]:
visual.visual_strain()

### Visualize MAE comparison 

In [ ]:
visual.visual_diff(label_style='b')

In [22]:
visual.record_performance(data_index=True,
                          data_index_path=f'{folder_name}/simulated_4dstem_grain_index.npy',
                          show_index_map=False,
                          width=2)

In [23]:
visual.add_data_2_plot()

# 05% Noisy 4D STEM

## Load pretraiend weight and rotation path

In [25]:
bkg_intensity = 0.05
bkg_str = format(int(bkg_intensity*100),'02d')
rotation_path = f'{folder_name}/{bkg_str}Percent_pretrained_rotation.npy'

## Update train class parameter for data generating 

In [26]:
tc.background_weight= bkg_intensity
tc.learned_rotation = rotation_path
# add extra rotation accoding to training strategy
tc.adjust_learned_rotation = 0

## Generate dataset

In [ ]:
tc.reset_dataset()

## Load pretrained weight to neural network

In [28]:
weight_path = f"{folder_name}/{bkg_str}percent_noisy_simulated_4dstem_pretrained_weights.pkl"
tc.load_pretrained_weight(weight_path)

In [ ]:
tc.predict(train_process='2',
           save_strain = True,
           save_rotation = True,
           file_name=bkg_intensity)

## Update visual result class
### Set hyper parameters of the class

In [24]:
bkg_intensity = 0.05
bkg_str = format(int(bkg_intensity*100),'02d')
rotation_path = f'{folder_name}/{bkg_str}Per_2_train_process_rotation.npy'
strain_path = f'{folder_name}/{bkg_str}Per_2_train_process_scale_shear.npy'
rotation_5per = np.load(rotation_path)
scale_shear_5per = np.load(strain_path)

### Load variable into visualization class

In [25]:
visual.file_py4DSTEM = f'{folder_name}/analysis_bg{bkg_str}per_1e5counts__strain.h5'
visual.rotation = rotation_5per
visual.scale_shear = scale_shear_5per
visual.noise_intensity = bkg_intensity

### Reset baseline of py4DSTEM

In [26]:
visual.reset_baseline()

### Shift angle to compare appropriately

In [ ]:
visual.reset_angle(angle_shift=-7)

### Update polar decomposition based on new affine matrix

In [ ]:
visual.reset_polar_matrix()

### Visualize strain map result

In [ ]:
visual.visual_strain()

### Visualize MAE comparison

In [ ]:
visual.visual_diff()

In [31]:
visual.record_performance(data_index=True,
                          data_index_path=f'{folder_name}/simulated_4dstem_grain_index.npy',
                          show_index_map=False,
                          width=2)

In [32]:
visual.add_data_2_plot()

# 10% Noisy 4D STEM

## Load pretraiend weight and rotation path

In [39]:
bkg_intensity = 0.10
bkg_str = format(int(bkg_intensity*100),'02d')
rotation_path = f'{folder_name}/{bkg_str}Percent_pretrained_rotation.npy'

### Add extra rotation accoding to training strategy

In [40]:
tc.background_weight= bkg_intensity
tc.learned_rotation = rotation_path
# add extra rotation accoding to training strategy
tc.adjust_learned_rotation = -25

## Generate dataset

In [ ]:
tc.reset_dataset()

## Load pretrained weight to neural network

In [42]:
weight_path = f"{folder_name}/{bkg_str}percent_noisy_simulated_4dstem_pretrained_weights.pkl"
tc.load_pretrained_weight(weight_path)

In [ ]:
tc.predict(train_process='2',
           save_strain = True,
           save_rotation = True,
           file_name=bkg_intensity)

## Update visual result class
### Set hyper parameters of the class

In [33]:
bkg_intensity = 0.10
bkg_str = format(int(bkg_intensity*100),'02d')
rotation_path = f'{folder_name}/{bkg_str}Per_2_train_process_rotation.npy'
strain_path = f'{folder_name}/{bkg_str}Per_2_train_process_scale_shear.npy'
rotation_10per = np.load(rotation_path)
scale_shear_10per = np.load(strain_path)

### Load variable into visualization class

In [34]:
visual.file_py4DSTEM = f'{folder_name}/analysis_bg{bkg_str}per_1e5counts__strain.h5'
visual.rotation = rotation_10per
visual.scale_shear = scale_shear_10per
visual.noise_intensity = bkg_intensity

### Reset baseline of py4DSTEM

In [35]:
visual.reset_baseline()

### Shift angle to compare appropriately

In [ ]:
visual.reset_angle(angle_shift=-5)

### Update polar decomposition based on new affine matrix

In [ ]:
visual.reset_polar_matrix()

### Visualize strain result

In [ ]:
visual.visual_strain()

### visualize MAE comparison

In [ ]:
visual.visual_diff()

In [40]:
visual.record_performance(data_index=True,
                          data_index_path=f'{folder_name}/simulated_4dstem_grain_index.npy',
                          show_index_map=False,
                          width=2)

In [41]:
visual.add_data_2_plot()

# 15% Noisy 4D STEM

## Load pretraiend weight and rotation path

In [53]:
bkg_intensity = 0.15
bkg_str = format(int(bkg_intensity*100),'02d')
rotation_path = f'{folder_name}/{bkg_str}Percent_pretrained_rotation.npy'

### Add extra rotation accoding to training strategy

In [54]:
tc.background_weight= bkg_intensity
tc.learned_rotation = rotation_path
# add extra rotation accoding to training strategy
tc.adjust_learned_rotation = 0

## Generate dataset

In [ ]:
tc.reset_dataset()

## Load pretrained weight to neural network

In [56]:
weight_path = f"{folder_name}/{bkg_str}percent_noisy_simulated_4dstem_pretrained_weights.pkl"
tc.load_pretrained_weight(weight_path)

In [ ]:
tc.predict(train_process='2',
           save_strain = True,
           save_rotation = True,
           file_name=bkg_intensity)

## Update visual result class
### Set hyper parameters of the class

In [42]:
bkg_intensity = 0.15
bkg_str = format(int(bkg_intensity*100),'02d')
rotation_path = f'{folder_name}/{bkg_str}Per_2_train_process_rotation.npy'
strain_path = f'{folder_name}/{bkg_str}Per_2_train_process_scale_shear.npy'
rotation_15per = np.load(rotation_path)
scale_shear_15per = np.load(strain_path)

### Load variable into visualization class

In [43]:
visual.file_py4DSTEM = f'{folder_name}/analysis_bg{bkg_str}per_1e5counts__strain.h5'
visual.rotation = rotation_15per
visual.scale_shear = scale_shear_15per
visual.noise_intensity = bkg_intensity

### Reset baseline of py4DSTEM

In [44]:
visual.reset_baseline()

### Shift angle to compare appropriately

In [ ]:
visual.reset_angle(angle_shift=-8)

### Update polar decomposition based on new affine matrix

In [ ]:
visual.reset_polar_matrix()

### Visualize strain result

In [ ]:
visual.visual_strain()

### visualize MAE comparison

In [ ]:
visual.visual_diff()

In [49]:
visual.record_performance(data_index=True,
                          data_index_path=f'{folder_name}/simulated_4dstem_grain_index.npy',
                          show_index_map=False,
                          width=2)

In [50]:
visual.add_data_2_plot()

#  20% Noisy 4D STEM 

## Load pretraiend weight and rotation path

In [ ]:
bkg_intensity = 0.20
bkg_str = format(int(bkg_intensity*100),'02d')
rotation_path = f'{folder_name}/{bkg_str}Percent_pretrained_rotation.npy'

### Add extra rotation accoding to training strategy

In [ ]:
tc.background_weight= bkg_intensity
tc.learned_rotation = rotation_path
# add extra rotation accoding to training strategy
tc.adjust_learned_rotation = -20

## Generate dataset

In [ ]:
tc.reset_dataset()

## Load pretrained weight to neural network

In [ ]:
weight_path = f"{folder_name}/{bkg_str}percent_noisy_simulated_4dstem_pretrained_weights.pkl"
tc.load_pretrained_weight(weight_path)

In [ ]:
tc.predict(train_process='2',
           save_strain = True,
           save_rotation = True,
           file_name=bkg_intensity)

## Update visual result class
### Set hyper parameters of the class

In [51]:
bkg_intensity = 0.20
bkg_str = format(int(bkg_intensity*100),'02d')
rotation_path = f'{folder_name}/{bkg_str}Per_2_train_process_rotation.npy'
strain_path = f'{folder_name}/{bkg_str}Per_2_train_process_scale_shear.npy'
rotation_20per = np.load(rotation_path)
scale_shear_20per = np.load(strain_path)

### Load variable into visualization class

In [52]:
visual.file_py4DSTEM = f'{folder_name}/analysis_bg{bkg_str}per_1e5counts__strain.h5'
visual.rotation = rotation_20per
visual.scale_shear = scale_shear_20per
visual.noise_intensity = bkg_intensity

### Reset baseline of py4DSTEM

In [53]:
visual.reset_baseline()

### Shift angle to compare appropriately

In [ ]:
visual.reset_angle(angle_shift=-7)

### Update polar decomposition based on new affine matrix

In [ ]:
visual.reset_polar_matrix()

### Visualize strain result

In [ ]:
visual.visual_strain()

### Visualize MAE comparison

In [ ]:
visual.visual_diff()

In [58]:
visual.record_performance(data_index=True,
                          data_index_path=f'{folder_name}/simulated_4dstem_grain_index.npy',
                          show_index_map=False,
                          width=2)

In [59]:
visual.add_data_2_plot()

# 25% Noisy 4D STEM

## Load pretraiend weight and rotation path

In [ ]:
bkg_intensity = 0.25
bkg_str = format(int(bkg_intensity*100),'02d')
rotation_path = f'{folder_name}/{bkg_str}Percent_pretrained_rotation.npy'

### Add extra rotation accoding to training strategy

In [ ]:
tc.background_weight= bkg_intensity
tc.learned_rotation = rotation_path
# add extra rotation accoding to training strategy
tc.adjust_learned_rotation = -15

## Generate dataset

In [ ]:
tc.reset_dataset()

## Load pretrained weight to neural network

In [ ]:
weight_path = f"{folder_name}/{bkg_str}percent_noisy_simulated_4dstem_pretrained_weights.pkl"
tc.load_pretrained_weight(weight_path)

In [ ]:
tc.predict(train_process='2',
           save_strain = True,
           save_rotation = True,
           file_name=bkg_intensity)

## Update visual result class
### Set hyper parameters of the class

In [60]:
bkg_intensity = 0.25
bkg_str = format(int(bkg_intensity*100),'02d')
rotation_path = f'{folder_name}/{bkg_str}Per_2_train_process_rotation.npy'
strain_path = f'{folder_name}/{bkg_str}Per_2_train_process_scale_shear.npy'
rotation_25per = np.load(rotation_path)
scale_shear_25per = np.load(strain_path)

### Load variable into visualization class

In [61]:
visual.file_py4DSTEM = f'{folder_name}/analysis_bg{bkg_str}per_1e5counts__strain.h5'
visual.rotation = rotation_25per
visual.scale_shear = scale_shear_25per
visual.noise_intensity = bkg_intensity

### Reset baseline of py4DSTEM

In [62]:
visual.reset_baseline()

### Shift angle to compare appropriately

In [ ]:
visual.reset_angle(angle_shift=-9)

### Update polar decomposition based on new affine matrix

In [ ]:
visual.reset_polar_matrix()

### Visualize strain result

In [ ]:
visual.visual_strain()

### Visualize MAE comparison

In [ ]:
visual.visual_diff()

In [67]:
visual.record_performance(data_index=True,
                          data_index_path=f'{folder_name}/simulated_4dstem_grain_index.npy',
                          show_index_map=False,
                          width=2)

In [68]:
visual.add_data_2_plot()

# Visualize 30Percent Background

## Load pretraiend weight and rotation path

In [ ]:
bkg_intensity = 0.30
bkg_str = format(int(bkg_intensity*100),'02d')
rotation_path = f'{folder_name}/{bkg_str}Percent_pretrained_rotation.npy'

### Add extra rotation accoding to training strategy

In [ ]:
tc.background_weight= bkg_intensity
tc.learned_rotation = rotation_path
# add extra rotation accoding to training strategy
tc.adjust_learned_rotation = -30

## Generate dataset

In [ ]:
tc.reset_dataset()

## Load pretrained weight to neural network

In [ ]:
weight_path = f"{folder_name}/{bkg_str}percent_noisy_simulated_4dstem_pretrained_weights.pkl"
tc.load_pretrained_weight(weight_path)

In [ ]:
tc.predict(train_process='2',
           save_strain = True,
           save_rotation = True,
           file_name=bkg_intensity)

## Update visual result class
### Set hyper parameters of the class

In [69]:
bkg_intensity = 0.30
bkg_str = format(int(bkg_intensity*100),'02d')
rotation_path = f'{folder_name}/{bkg_str}Per_2_train_process_rotation.npy'
strain_path = f'{folder_name}/{bkg_str}Per_2_train_process_scale_shear.npy'
rotation_30per = np.load(rotation_path)
scale_shear_30per = np.load(strain_path)

### Load variable into visualization class

In [70]:
visual.file_py4DSTEM = f'{folder_name}/analysis_bg{bkg_str}per_1e5counts__strain.h5'
visual.rotation = rotation_30per
visual.scale_shear = scale_shear_30per
visual.noise_intensity = bkg_intensity

### Reset baseline of py4DSTEM

In [71]:
visual.reset_baseline()

### Shift angle to compare appropriately

In [ ]:
visual.reset_angle(angle_shift=-9)

### Update polar decomposition based on new affine matrix

In [ ]:
visual.reset_polar_matrix()

### Visualize strain result

In [ ]:
visual.visual_strain()

### Visualize MAE comparison

In [ ]:
visual.visual_diff()

In [76]:
visual.record_performance(data_index=True,
                          data_index_path=f'{folder_name}/simulated_4dstem_grain_index.npy',
                          show_index_map=False,
                          width=2)

In [77]:
visual.add_data_2_plot()

# Visualize 35Percent Background

## Load pretraiend weight and rotation path

In [ ]:
bkg_intensity = 0.35
bkg_str = format(int(bkg_intensity*100),'02d')
rotation_path = f'{folder_name}/{bkg_str}Percent_pretrained_rotation.npy'

### Add extra rotation accoding to training strategy

In [ ]:
tc.background_weight= bkg_intensity
tc.learned_rotation = rotation_path
# add extra rotation accoding to training strategy
tc.adjust_learned_rotation = -5

## Generate dataset

In [ ]:
tc.reset_dataset()

## Load pretrained weight to neural network

In [ ]:
weight_path = f"{folder_name}/{bkg_str}percent_noisy_simulated_4dstem_pretrained_weights.pkl"
tc.load_pretrained_weight(weight_path)

In [ ]:
tc.predict(train_process='2',
           save_strain = True,
           save_rotation = True,
           file_name=bkg_intensity)

## Update visual result class
### Set hyper parameters of the class

In [78]:
bkg_intensity = 0.35
bkg_str = format(int(bkg_intensity*100),'02d')
rotation_path = f'{folder_name}/{bkg_str}Per_2_train_process_rotation.npy'
strain_path = f'{folder_name}/{bkg_str}Per_2_train_process_scale_shear.npy'
rotation_35per = np.load(rotation_path)
scale_shear_35per = np.load(strain_path)

### Load variable into visualization class

In [79]:
visual.file_py4DSTEM = f'{folder_name}/analysis_bg{bkg_str}per_1e5counts__strain.h5'
visual.rotation = rotation_35per
visual.scale_shear = scale_shear_35per
visual.noise_intensity = bkg_intensity

### Reset baseline of py4DSTEM

In [80]:
visual.reset_baseline()

### Shift angle to compare appropriately

In [ ]:
visual.reset_angle(angle_shift=-6)

### Update polar decomposition based on new affine matrix

In [ ]:
visual.reset_polar_matrix()

### Visualize strain result

In [ ]:
visual.visual_strain()

### visualize MAE comparison

In [ ]:
visual.visual_diff()

In [85]:
visual.record_performance(data_index=True,
                          data_index_path=f'{folder_name}/simulated_4dstem_grain_index.npy',
                          show_index_map=False,
                          width=2)

In [86]:
visual.add_data_2_plot()

# Visualize 40Percent Background

## Load pretraiend weight and rotation path

In [ ]:
bkg_intensity = 0.40
bkg_str = format(int(bkg_intensity*100),'02d')
rotation_path = f'{folder_name}/{bkg_str}Percent_pretrained_rotation.npy'

### Add extra rotation accoding to training strategy

In [ ]:
tc.background_weight= bkg_intensity
tc.learned_rotation = rotation_path
# add extra rotation accoding to training strategy
tc.adjust_learned_rotation = -25

## Generate dataset

In [ ]:
tc.reset_dataset()

## Load pretrained weight to neural network

In [ ]:
weight_path = f"{folder_name}/{bkg_str}percent_noisy_simulated_4dstem_pretrained_weights.pkl"
tc.load_pretrained_weight(weight_path)

In [ ]:
tc.predict(train_process='2',
           save_strain = True,
           save_rotation = True,
           file_name=bkg_intensity)

## Update visual result class
### Set hyper parameters of the class

In [87]:
bkg_intensity = 0.40
bkg_str = format(int(bkg_intensity*100),'02d')
rotation_path = f'{folder_name}/{bkg_str}Per_2_train_process_rotation.npy'
strain_path = f'{folder_name}/{bkg_str}Per_2_train_process_scale_shear.npy'
rotation_40per = np.load(rotation_path)
scale_shear_40per = np.load(strain_path)

### Load variable into visualization class

In [88]:
visual.file_py4DSTEM = f'{folder_name}/analysis_bg{bkg_str}per_1e5counts__strain.h5'
visual.rotation = rotation_40per
visual.scale_shear = scale_shear_40per
visual.noise_intensity = bkg_intensity

### Reset baseline of py4DSTEM

In [89]:
visual.reset_baseline()

### Shift angle to compare appropriately

In [ ]:
visual.reset_angle(angle_shift=-8)

### Update polar decomposition based on new affine matrix

In [ ]:
visual.reset_polar_matrix()

### Visualize strain result

In [ ]:
visual.visual_strain()

### visualize MAE comparison

In [ ]:
visual.visual_diff()

In [94]:
visual.record_performance(data_index=True,
                          data_index_path=f'{folder_name}/simulated_4dstem_grain_index.npy',
                          show_index_map=False,
                          width=2)

In [95]:
visual.add_data_2_plot()

# Visualize 45Percent Background

## Load pretraiend weight and rotation path

In [ ]:
bkg_intensity = 0.45
bkg_str = format(int(bkg_intensity*100),'02d')
rotation_path = f'{folder_name}/{bkg_str}Percent_pretrained_rotation.npy'

### Add extra rotation accoding to training strategy

In [ ]:
tc.background_weight= bkg_intensity
tc.learned_rotation = rotation_path
# add extra rotation accoding to training strategy
tc.adjust_learned_rotation = -40

## Generate dataset

In [ ]:
tc.reset_dataset()

## Load pretrained weight to neural network

In [ ]:
weight_path = f"{folder_name}/{bkg_str}percent_noisy_simulated_4dstem_pretrained_weights.pkl"
tc.load_pretrained_weight(weight_path)

In [ ]:
tc.predict(train_process='2',
           save_strain = True,
           save_rotation = True,
           file_name=bkg_intensity)

## Update visual result class
### Set hyper parameters of the class

In [96]:
bkg_intensity = 0.45
bkg_str = format(int(bkg_intensity*100),'02d')
rotation_path = f'{folder_name}/{bkg_str}Per_2_train_process_rotation.npy'
strain_path = f'{folder_name}/{bkg_str}Per_2_train_process_scale_shear.npy'
rotation_45per = np.load(rotation_path)
scale_shear_45per = np.load(strain_path)

### Load variable into visualization class

In [97]:
visual.file_py4DSTEM = f'{folder_name}/analysis_bg{bkg_str}per_1e5counts__strain.h5'
visual.rotation = rotation_45per
visual.scale_shear = scale_shear_45per
visual.noise_intensity = bkg_intensity

### Reset baseline of py4DSTEM

In [98]:
visual.reset_baseline()

### Shift angle to compare appropriately

In [ ]:
visual.reset_angle(angle_shift=-7)

### Update polar decomposition based on new affine matrix

In [ ]:
visual.reset_polar_matrix()

### Visualize strain result

In [ ]:
visual.visual_strain()

### visualize MAE comparison

In [ ]:
visual.visual_diff()

In [103]:
visual.record_performance(data_index=True,
                          data_index_path=f'{folder_name}/simulated_4dstem_grain_index.npy',
                          show_index_map=False,
                          width=2)

In [104]:
visual.add_data_2_plot()

# Visualize 50Percent Background

## Load pretraiend weight and rotation path

In [ ]:
bkg_intensity = 0.50
bkg_str = format(int(bkg_intensity*100),'02d')
rotation_path = f'{folder_name}/{bkg_str}Percent_pretrained_rotation.npy'

### Add extra rotation accoding to training strategy

In [ ]:
tc.background_weight= bkg_intensity
tc.learned_rotation = rotation_path
# add extra rotation accoding to training strategy
tc.adjust_learned_rotation = -20

## Generate dataset

In [ ]:
tc.reset_dataset()

## Load pretrained weight to neural network

In [ ]:
weight_path = f"{folder_name}/{bkg_str}percent_noisy_simulated_4dstem_pretrained_weights.pkl"
tc.load_pretrained_weight(weight_path)

In [ ]:
tc.predict(train_process='2',
           save_strain = True,
           save_rotation = True,
           file_name=bkg_intensity)

## Update visual result class
### Set hyper parameters of the class

In [105]:
bkg_intensity = 0.50
bkg_str = format(int(bkg_intensity*100),'02d')
rotation_path = f'{folder_name}/{bkg_str}Per_2_train_process_rotation.npy'
strain_path = f'{folder_name}/{bkg_str}Per_2_train_process_scale_shear.npy'
rotation_50per = np.load(rotation_path)
scale_shear_50per = np.load(strain_path)

### Load variable into visualization class

In [106]:
visual.file_py4DSTEM = f'{folder_name}/analysis_bg{bkg_str}per_1e5counts__strain.h5'
visual.rotation = rotation_50per
visual.scale_shear = scale_shear_50per
visual.noise_intensity = bkg_intensity

### Reset baseline of py4DSTEM

In [107]:
visual.reset_baseline()

### Shift angle to compare appropriately

In [ ]:
visual.reset_angle(angle_shift=-6)

### Update polar decomposition based on new affine matrix

In [ ]:
visual.reset_polar_matrix()

### Visualize strain result

In [ ]:
visual.visual_strain()

### Visualize MAE comparison

In [ ]:
visual.visual_diff()

In [112]:
visual.record_performance(data_index=True,
                          data_index_path=f'{folder_name}/simulated_4dstem_grain_index.npy',
                          show_index_map=False,
                          width=2)

In [113]:
visual.add_data_2_plot()

# Visualize 60Percent Background

## Load pretraiend weight and rotation path

In [ ]:
bkg_intensity = 0.60
bkg_str = format(int(bkg_intensity*100),'02d')
rotation_path = f'{folder_name}/{bkg_str}Percent_pretrained_rotation.npy'

### Add extra rotation accoding to training strategy

In [ ]:
tc.background_weight= bkg_intensity
tc.learned_rotation = rotation_path
# add extra rotation accoding to training strategy
tc.adjust_learned_rotation = 1

## Generate dataset

In [ ]:
tc.reset_dataset()

## Load pretrained weight to neural network

In [ ]:
weight_path = f"{folder_name}/{bkg_str}percent_noisy_simulated_4dstem_pretrained_weights.pkl"
tc.load_pretrained_weight(weight_path)

In [ ]:
tc.predict(train_process='2',
           save_strain = True,
           save_rotation = True,
           file_name=bkg_intensity)

## Update visual result class
### Set hyper parameters of the class

In [114]:
bkg_intensity = 0.60
bkg_str = format(int(bkg_intensity*100),'02d')
rotation_path = f'{folder_name}/{bkg_str}Per_2_train_process_rotation.npy'
strain_path = f'{folder_name}/{bkg_str}Per_2_train_process_scale_shear.npy'
rotation_60per = np.load(rotation_path)
scale_shear_60per = np.load(strain_path)

### Load variable into visualization class

In [115]:
visual.file_py4DSTEM = f'{folder_name}/analysis_bg{bkg_str}per_1e5counts__strain.h5'
visual.rotation = rotation_60per
visual.scale_shear = scale_shear_60per
visual.noise_intensity = bkg_intensity

### Reset baseline of py4DSTEM

In [116]:
visual.reset_baseline()

### Shift angle to compare appropriately

In [ ]:
visual.reset_angle(angle_shift=-9)

### Update polar decomposition based on new affine matrix

In [ ]:
visual.reset_polar_matrix()

### Visualize strain result

In [ ]:
visual.visual_strain()

### Visualize MAE comparison

In [ ]:
visual.mae_diff_range = [-0.01,0.01]
visual.visual_diff()

In [121]:
visual.record_performance(data_index=True,
                          data_index_path=f'{folder_name}/simulated_4dstem_grain_index.npy',
                          show_index_map=False,
                          width=2)

In [122]:
visual.add_data_2_plot()

# Visualize 70Percent Background

## Load pretraiend weight and rotation path

In [ ]:
bkg_intensity = 0.70
bkg_str = format(int(bkg_intensity*100),'02d')
rotation_path = f'{folder_name}/{bkg_str}Percent_pretrained_rotation.npy'

### Add extra rotation accoding to training strategy

In [ ]:
tc.background_weight= bkg_intensity
tc.learned_rotation = rotation_path
# add extra rotation accoding to training strategy
tc.adjust_learned_rotation = 20

## Generate dataset

In [ ]:
tc.reset_dataset()

## Load pretrained weight to neural network

In [ ]:
weight_path = f"{folder_name}/{bkg_str}percent_noisy_simulated_4dstem_pretrained_weights.pkl"
tc.load_pretrained_weight(weight_path)

In [ ]:
tc.predict(train_process='2',
           save_strain = True,
           save_rotation = True,
           file_name=bkg_intensity)

## Update visual result class
### Set hyper parameters of the class

In [123]:
bkg_intensity = 0.70
bkg_str = format(int(bkg_intensity*100),'02d')
rotation_path = f'{folder_name}/{bkg_str}Per_2_train_process_rotation.npy'
strain_path = f'{folder_name}/{bkg_str}Per_2_train_process_scale_shear.npy'
rotation_70per = np.load(rotation_path)
scale_shear_70per = np.load(strain_path)

### Load variable into visualization class

In [124]:
visual.file_py4DSTEM = f'{folder_name}/analysis_bg{bkg_str}per_1e5counts__strain.h5'
visual.rotation = rotation_70per
visual.scale_shear = scale_shear_70per
visual.noise_intensity = bkg_intensity

### Reset baseline of py4DSTEM

In [125]:
visual.reset_baseline()

### Shift angle to compare appropriately

In [ ]:
visual.reset_angle(angle_shift=-8)

### Update polar decomposition based on new affine matrix

In [ ]:
visual.reset_polar_matrix()

### Visualize strain result

In [ ]:
visual.visual_strain()

### Visualize MAE comparison

In [ ]:
visual.mae_diff_range = [-0.015,0.015]
visual.visual_diff()

In [130]:
visual.record_performance(data_index=True,
                          data_index_path=f'{folder_name}/simulated_4dstem_grain_index.npy',
                          show_index_map=False,
                          width=2)

In [131]:
visual.add_data_2_plot()

## Visualize performance of both methods

In [132]:
img_show = True
if img_show:
    %matplotlib inline
else:
    plt.ioff()

In [133]:
direct_load = True
if direct_load:
    with open(f'{folder_name}/MAE_list_simulated_4dstem.pickle', 'rb') as pickle_file:
        visual.list_of_dic = pickle.load(pickle_file)

### Generate various noise level plots of both methods
### Generate Strain XX results

In [134]:
# maually_record_fpga_results

# x_add = [0,0.25,0.6]
# fpga_xx = [0.002,0.0017,0.0019]
# fpga_yy = [0.0026,0.0017,0.0022]
# fpga_xy = [0.0015,0.0011,0.0016]
# fpga_rot = [1.63,1.28,1.47]

In [ ]:
visual_performance_plot(x_list=visual.extract_ele_from_dic(0)[0],
                        auto=visual.extract_ele_from_dic(4)[1],
                        py4d=visual.extract_ele_from_dic(0)[1],
#                         auto_yerr =visual.extract_ele_from_dic(12)[1],
#                         py4d_yerr =visual.extract_ele_from_dic(8)[1],
                        title = 'Strian X',
                        xlabel = '',
                        ylabel ='',
                        # add_x = x_add,
                        # add_y = fpga_xx,
                        marker_list = ['o','H','^'],
                        figsize=(6,5),
                        ylim = [0,0.005],
                        fill_between = False,
                        errorbar= False,
                        folder_path='save_figures/')

### Generate Strain YY results

In [ ]:
visual_performance_plot(x_list=visual.extract_ele_from_dic(0)[0],
                        auto=visual.extract_ele_from_dic(5)[1],
                        py4d=visual.extract_ele_from_dic(1)[1],
                        # auto_yerr =visual.extract_ele_from_dic(13)[1],
                        # py4d_yerr =visual.extract_ele_from_dic(9)[1],
                        title = 'Strian Y',
                        xlabel = '',
                        ylabel ='',
                        # add_x = x_add,
                        # add_y = fpga_yy,
                        marker_list = ['o','H','^'],
                        figsize=(6,5),
                        ylim = [0,0.005],
                        fill_between = False,
                        errorbar=False,
                        folder_path='save_figures/')

### Generate Strain XY results

In [ ]:
visual_performance_plot(x_list=visual.extract_ele_from_dic(0)[0],
                        auto=visual.extract_ele_from_dic(6)[1],
                        py4d=visual.extract_ele_from_dic(2)[1],
                        # auto_yerr =visual.extract_ele_from_dic(14)[1],
                        # py4d_yerr =visual.extract_ele_from_dic(10)[1],
                        title = 'Shear',
                        xlabel = '',
                        ylabel ='',
                        # add_x = x_add,
                        # add_y = fpga_xy,
                        marker_list = ['o','H','^'],
                        figsize=(6,5),
                        ylim = [0,0.004],
                        fill_between = False,
                        errorbar=False,
                        folder_path='save_figures/')

### Generate Rotation results

In [ ]:
visual_performance_plot(x_list=visual.extract_ele_from_dic(0)[0],
                        auto=visual.extract_ele_from_dic(7)[1],
                        py4d=visual.extract_ele_from_dic(3)[1],
                        # auto_yerr =visual.extract_ele_from_dic(15)[1],
                        # py4d_yerr =visual.extract_ele_from_dic(11)[1],
                        title = 'Rotation',
                        xlabel = '',
                        ylabel ='',
                        # add_x = x_add,
                        # add_y = fpga_rot,
                        marker_list = ['o','H','^'],
                        figsize=(6,5),
                        ylim = [0,1.75],
                        fill_between = False,
                        errorbar= False,
                        folder_path='save_figures/')